# Importowanie bibliotek
W projekcie wykorzystano następujące biblioteki:
- TensorFlow i Keras: Do budowy i trenowania modelu głębokiego uczenia.
- NumPy i Pandas: Do manipulacji danymi i ich wstępnego przetwarzania.
- Scikit-learn: Do skalowania danych oraz podziału na zbiory treningowe i testowe.
- Keras Tuner: Do optymalizacji hiperparametrów modelu.
- Callbacks w Keras: EarlyStopping i ReduceLROnPlateau zapewniają zatrzymanie treningu w odpowiednim momencie i dostosowywanie tempa uczenia.

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras_tuner as kt
from tensorflow.keras.layers import Input

# Wczytanie i przygotowanie danych
Źródło danych: Dane zostały wczytane z pliku CSV (data.csv).

Filtracja cech: Wybrano istotne numeryczne cechy do modelowania regresji:
- Size(L) - rozmiar partii piwa.
- OG, FG - gęstość początkowa i końcowa.
- IBU - jednostki goryczki.
- Color - barwa piwa.
- BoilSize, BoilTime, BoilGravity, Efficiency - parametry procesu warzenia.

Uzupełnianie braków danych: Wszystkie brakujące wartości zostały zastąpione średnią dla każdej cechy.

Normalizacja danych: Dane wejściowe zostały znormalizowane przy użyciu StandardScaler, aby wszystkie cechy miały porównywalne 
zakresy wartości.

Podział danych: Dane podzielono na zbiór treningowy (80%) i testowy (20%).

In [3]:
data = pd.read_csv('../data.csv')

numeric_columns = ['Size(L)', 'OG', 'FG', 'IBU', 'Color', 'BoilSize', 'BoilTime', 'BoilGravity', 'Efficiency']
X = data[numeric_columns].copy()

X = X.fillna(X.mean())

y = data['ABV'].copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Struktura modelu
Zbudowano model sekwencyjny z następującymi warstwami:

Warstwa wejściowa:
- Liczba neuronów dynamicznie dobierana z przedziału 128–512.
- Funkcja aktywacji: ReLU.
Warstwy ukryte:
- Pierwsza warstwa: 256 neuronów, aktywacja ReLU.
- Druga warstwa: 112 neuronów, aktywacja ReLU.
- Trzecia warstwa: 48 neuronów, aktywacja ReLU.
- Każda warstwa ukryta zawierała:
  - Batch Normalization: Stabilizacja procesu uczenia poprzez normalizację wsadową.
  - Dropout: Dynamicznie dobierany (20–70%), aby zapobiec przeuczeniu modelu.
  
Warstwa wyjściowa:
- Jeden neuron, który zwraca przewidywaną wartość regresji (zawartość alkoholu).

Kompilacja modelu
- Optymalizator: Adam, dynamicznie dobierana szybkość uczenia (0.0001–0.01).
- Funkcja straty: mean_squared_error (średni błąd kwadratowy).
- Metryka: mean_absolute_error (średni błąd bezwzględny).

In [4]:
units_layer_1_list = [128, 160, 192, 224, 256, 288, 320, 352, 384, 416, 448, 480, 512]
units_layer_2_list = [64, 96, 128, 160, 192, 224, 256]
units_layer_3_list = [32, 48, 64, 80, 96, 112, 128]
units_layer_4_list = [16, 32, 48, 64]
dropout_list = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
learning_rate_list = [1e-4, 3e-4, 1e-3, 3e-3, 1e-2]
batch_size_list = [16, 32, 48, 64, 80, 96, 112, 128]

def build_model(hp):
    model = Sequential()

    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(
        units=hp.Choice('units_layer_1', values=units_layer_1_list),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_1', values=dropout_list))) 

    model.add(Dense(
        units=hp.Choice('units_layer_2', values=units_layer_2_list), 
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_2', values=dropout_list))) 

    model.add(Dense(
        units=hp.Choice('units_layer_3', values=units_layer_3_list), 
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_3', values=dropout_list))) 

    model.add(Dense(
        units=hp.Choice('units_layer_4', values=units_layer_4_list), 
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Choice('dropout_layer_4', values=dropout_list))) 

    model.add(Dense(1))

    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=learning_rate_list)), 
        loss='mean_squared_error',
        metrics=['mean_absolute_error']
    )
    
    return model


# Proces tuningu hiperparametrów
Metoda tuningu: Bayesian Optimization (optymalizacja bayesowska).

Zakres testowanych hiperparametrów:
- Liczba neuronów w każdej warstwie:
  - Warstwa 1: 128–512 neuronów.
  - Warstwa 2: 64–256 neuronów.
  - Warstwa 3: 32–128 neuronów.
  - Warstwa 4: 16–64 neuronów.
- Dropout: 0.2–0.7 (krok 0.1).
- Learning rate: 0.0001–0.01 (skala logarytmiczna).
- Batch size: 16–128.

Liczba prób: 20 (maksymalna liczba konfiguracji).

Proces uczenia
- Parametry treningu:
  - Maksymalna liczba epok: 20.
  - Batch size: Dynamicznie dobierany, w przykładzie 32.
- Callbacki:
  - EarlyStopping: Monitorowanie straty walidacji (val_loss), zatrzymanie po 5 epokach bez poprawy.
  - TensorBoard: Monitorowanie wyników uczenia w czasie rzeczywistym.

In [5]:
tuner = kt.BayesianOptimization(build_model,
    objective='val_mean_absolute_error',
    max_trials=20,
    directory='logs/fit',
    project_name='regresja_alkoholu'
)

early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=5, 
                               restore_best_weights=True)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/fit/regresja_alkoholu')

tuner.search(X_train, y_train, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[early_stopping, tensorboard_callback],
             batch_size=32)

Trial 20 Complete [00h 00m 31s]
val_mean_absolute_error: 0.30087849497795105

Best val_mean_absolute_error So Far: 0.21853472292423248
Total elapsed time: 00h 13m 04s


In [6]:
best_model = tuner.get_best_models(num_models=1)[0]

best_model.summary()

c:\Users\Micha\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 160)            │        20,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 160)            │           640 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 160)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 96)             │        15,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 96)             │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,177 (164.75 KB)

 Trainable params: 41,345 (161.50 KB)

 Non-trainable params: 832 (3.25 KB)

# Wyniki

Najlepszy model: Osiągnął następujące wyniki na zbiorze walidacyjnym:
 - Średni błąd bezwzględny (MAE): 0.25
 - Średni błąd kwadratowy (MSE): strata walidacyjna stabilizowała się w zakresie 0.3–0.5 w większości epok.

Finalny model: Po zakończeniu treningu osiągnął MAE na zbiorze walidacyjnym na poziomie 0.42.

In [7]:
history = best_model.fit(
    X_train, y_train,
    epochs=20,
    validation_split=0.2,
    batch_size=32,
    callbacks=[early_stopping, tensorboard_callback],
)

best_model.summary()

final_val_mae = history.history['val_mean_absolute_error'][-1]
print(f"Final Validation MAE: {final_val_mae:.2f}")

Epoch 1/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.9849 - mean_absolute_error: 0.5480 - val_loss: 0.3573 - val_mean_absolute_error: 0.3348
Epoch 2/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.3193 - mean_absolute_error: 0.5782 - val_loss: 0.3510 - val_mean_absolute_error: 0.2783
Epoch 3/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.0481 - mean_absolute_error: 0.5559 - val_loss: 0.5147 - val_mean_absolute_error: 0.5084
Epoch 4/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.9725 - mean_absolute_error: 0.5471 - val_loss: 0.5647 - val_mean_absolute_error: 0.5082
Epoch 5/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.2833 - mean_absolute_error: 0.5737 - val_loss: 0.3180 - val_mean_absolute_error: 0.2506
Epoch 6/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.1356 - mean_absolute_error: 0.5554 - val_loss: 0.4640 - val_mean_absolute_error: 0.4270
Epoch 7/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.9936 - mean_absolute_err

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 160)            │        20,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 160)            │           640 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 160)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 96)             │        15,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 96)             │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 124,869 (487.77 KB)

 Trainable params: 41,345 (161.50 KB)

 Non-trainable params: 832 (3.25 KB)

 Optimizer params: 82,692 (323.02 KB)

Final Validation MAE: 0.42


# Zapisywanie Modelu

In [8]:
best_model.save('../models/tuned_model_regresji_alkoholu.h5')

loaded_model = keras.models.load_model('../models/tuned_model_regresji_alkoholu.h5')

loaded_model.summary()

loaded_test_loss, loaded_test_mae = loaded_model.evaluate(X_test, y_test)

print(f'Loaded Test Loss: {loaded_test_loss}')
print(f'Loaded Test Mean Absolute Error: {loaded_test_mae}')

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 160)            │        20,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 160)            │           640 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 160)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 96)             │        15,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 96)             │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,179 (164.77 KB)

 Trainable params: 41,345 (161.50 KB)

 Non-trainable params: 832 (3.25 KB)

 Optimizer params: 2 (12.00 B)

462/462 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step - loss: 0.4636 - mean_absolute_error: 0.3133
Loaded Test Loss: 0.6757606267929077
Loaded Test Mean Absolute Error: 0.3215130567550659
